In [14]:
import requests
import httpx 

In [15]:
path = "/api/events"
base_url = "http://localhost:8002"
endpoint = f"{base_url}{path}"
endpoint

'http://localhost:8002/api/events'

In [17]:
response = requests.get(endpoint)
print(response.status_code)

200
